In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import gensim
import nltk
from nltk.tokenize import word_tokenize
from modules.preprocess import *
from modules.utils import build_dataset, text_to_word2vec, evaluate
from modules.rnn_model import TextRNN, TextLSTM
import gensim.downloader as api
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

[nltk_data] Downloading package stopwords to /home/xavier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = build_dataset(path="lapresse_crawler", num_samples=100, rnd_state=10)

In [3]:
dataset = text_edit(dataset, grp_num=False, rm_newline=True, rm_punctuation=False,
              rm_stop_words=True, lowercase=True, lemmatize=False, html_=True, convert_entities=False, expand=True)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3630.21it/s]


In [4]:
X = [x['text'] for x in dataset.values() if x['section_1'] in ['actualites', 'sports', 'affaires', 'arts', 'international']]
Y = [x['section_label'] for x in dataset.values() if x['section_1'] in ['actualites', 'sports', 'affaires', 'arts', 'international']]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

In [6]:
#set([(x['section_1'], x['section_label']) for x in dataset.values()])

In [7]:
model_name = 'fasttext-wiki-news-subwords-300'  
word2vec_model = api.load(model_name)

In [8]:
text = "Ceci est un texte exemple"
vector = text_to_word2vec(text, word2vec_model)

In [9]:
input_size = vector.shape[0]  
hidden_size = 64
output_size = len(set(Y_train))  

In [ ]:
#model = TextRNN(input_size, hidden_size, output_size, batch_first=True, dropout=0, nonlinearity='relu', bidirectional=True, num_layers=2)
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model = TextLSTM(input_size, hidden_size, output_size, batch_first=True, dropout=0, bidirectional=False, num_layers=1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
X_train = torch.stack([torch.tensor(text_to_word2vec(x, word2vec_model), dtype=torch.float32).view(1,-1) for x in X_train], dim=0)
X_test = torch.stack([torch.tensor(text_to_word2vec(x, word2vec_model), dtype=torch.float32).view(1,-1) for x in X_test], dim=0)

In [ ]:
Y_train = torch.tensor(Y_train, dtype=torch.long)
Y_test = torch.tensor(Y_test, dtype=torch.long)

In [ ]:
batch_size = 32

dataset = TensorDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

best_test_loss = float('inf')
epochs = 50

for epoch in range(epochs):
    train_losses = []
    test_losses = []
    for X, Y in dataloader:  
        model.train()
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, Y)
        loss.backward() 
        optimizer.step()
        train_losses.append(loss.detach())
    for X, Y in test_dataloader:  
        model.eval()
        outputs = model(X)
        loss = criterion(outputs, Y)
        test_losses.append(loss.detach())

    mean_test_loss = np.mean(test_losses)
    print(f'Results for epoch {epoch}:')
    print(f'Mean train loss for epoch: {np.mean(train_losses)}')
    print(f'Mean test loss for epoch: {mean_test_loss}')

    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
        torch.save(model.state_dict(), 'rnn_best.pt') 
        print(f'Model saved at epoch {epoch} with test loss {mean_test_loss}')

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = TextRNN(input_size, hidden_size, output_size, batch_first=True, dropout=0, nonlinearity='relu', bidirectional=True, num_layers=2).to(device)  
#state_dict = torch.load('rnn_best.pt', map_location=device)  
#model.load_state_dict(state_dict)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextLSTM(input_size, hidden_size, output_size, batch_first=True, dropout=0, bidirectional=False, num_layers=1)
state_dict = torch.load('rnn_best.pt', map_location=device)  
model.load_state_dict(state_dict)

In [ ]:
model.eval()
pred_outputs = []
for tensor_ in X_test:
    output = model(tensor_.view(1,1,-1))
    pred_class = np.argmax(output.detach())
    pred_outputs.append(int(pred_class))

In [ ]:
evaluate(Y_test.numpy(), np.array(pred_outputs))